# 1 利用sklearn构建决策树

In [66]:
# 确定graphviz路径
import os
import math
import numpy as np
os.environ["PATH"] += os.pathsep + 'F:/Graphviz/bin/'

对鸢尾花数据集，构建CART决策树，并进行可视化：

（1）	导入数据集

（2）	CART决策树的构建与可视化

In [67]:
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split
import graphviz

In [68]:
# 加载鸢尾花数据集
iris = datasets.load_iris()
X = iris.data
y = iris.target

In [69]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [70]:
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

In [71]:
def visualize_tree(model, feature_names, class_names):
    """
    可视化决策树模型。

    参数:
    model: 决策树模型对象。
    feature_names: 特征名称列表。
    class_names: 类别名称列表。

    返回:
    graphviz.Source 对象，表示决策树的可视化表示。
    """
    # 使用 export_graphviz 函数将决策树模型转换为 DOT 格式的数据
    dot_data = export_graphviz(
        model,                # 决策树模型对象
        out_file=None,        # 输出文件路径，设置为 None 表示不输出到文件
        feature_names=feature_names,  # 特征名称列表
        class_names=class_names,      # 类别名称列表
        filled=True,          # 是否填充节点颜色
        rounded=True,         # 是否使用圆角节点
        special_characters=True  # 是否使用特殊字符
    )
    
    # 使用 graphviz.Source 创建决策树的可视化表示
    return graphviz.Source(dot_data)


In [72]:
# 可视化每棵树
tree_cart = visualize_tree(model, iris.feature_names, iris.target_names)

In [73]:
tree_cart.view()

'Source.gv.pdf'

# 2 手写实现决策树

In [74]:
import pandas as pd
import numpy as np
from graphviz import Digraph
from sklearn.metrics import accuracy_score

In [75]:
# 可视化决策树
def visualize_tree(tree, parent_name='', graph=None):
    # 如果没有传入 graph 对象，则创建一个新的 Digraph 对象，并设置输出格式为 'png'
    if graph is None:
        graph = graphviz.Digraph(format='png')
        graph.attr(fontname="SimHei")  # 设置字体为 SimHei 以支持中文显示
        graph.attr('node', shape='box')  # 设置节点形状为盒子

    # 遍历树的每个键值对
    for key, value in tree.items():
        # 生成当前节点的名称，如果有父节点，则在名称前加上父节点名称
        node_name = f'{parent_name}_{key}' if parent_name else key
        # 创建当前节点，并设置字体为 SimHei
        graph.node(node_name, key, fontname="SimHei")
        # 如果有父节点，则创建从父节点到当前节点的边，并设置字体为 SimHei
        if parent_name:
            graph.edge(parent_name, node_name, fontname="SimHei")

        # 如果当前值是字典，递归调用 visualize_tree 函数
        if isinstance(value, dict):
            visualize_tree(value, node_name, graph)
        else:
            # 如果当前值不是字典，说明是叶子节点，创建叶子节点并设置字体为 SimHei
            leaf_name = f'{node_name}_{value}'
            graph.node(leaf_name, str(value), fontname="SimHei")
            # 创建从当前节点到叶子节点的边，并设置字体为 SimHei
            graph.edge(node_name, leaf_name, fontname="SimHei")

    # 返回 graph 对象
    return graph


In [76]:
# 加载数据集
data = {
    '色泽': ['青绿', '乌黑', '乌黑', '青绿', '浅白', '青绿', '乌黑', '乌黑', '乌黑', '青绿', '浅白', '浅白', '青绿', '浅白', '乌黑', '浅白', '青绿'],
    '根蒂': ['蜷缩', '蜷缩', '蜷缩', '蜷缩', '蜷缩', '稍蜷', '稍蜷', '稍蜷', '稍蜷', '硬挺', '硬挺', '蜷缩', '稍蜷', '稍蜷', '稍蜷', '蜷缩', '蜷缩'],
    '敲声': ['浊响', '沉闷', '浊响', '沉闷', '浊响', '浊响', '浊响', '浊响', '沉闷', '清脆', '清脆', '浊响', '浊响', '沉闷', '浊响', '浊响', '沉闷'],
    '纹理': ['清晰', '清晰', '清晰', '清晰', '清晰', '清晰', '稍糊', '清晰', '稍糊', '清晰', '模糊', '模糊', '稍糊', '稍糊', '清晰', '模糊', '稍糊'],
    '脐部': ['凹陷', '凹陷', '凹陷', '凹陷', '凹陷', '稍凹', '稍凹', '稍凹', '稍凹', '平坦', '平坦', '平坦', '凹陷', '凹陷', '稍凹', '平坦', '稍凹'],
    '触感': ['硬滑', '硬滑', '硬滑', '硬滑', '硬滑', '软粘', '软粘', '硬滑', '硬滑', '软粘', '硬滑', '软粘', '硬滑', '硬滑', '软粘', '硬滑', '硬滑'],
    '好瓜': ['是', '是', '是', '是', '是', '是', '是', '是', '否', '否', '否', '否', '否', '否', '否', '否', '否']
}
df = pd.DataFrame(data)

In [77]:
# 特征和标签
X = df.drop('好瓜', axis=1)
y = df['好瓜'].apply(lambda x: 1 if x == '是' else 0)

## ID3

# 计算信息熵
$$
Ent(D) = -\sum_{k=1}^{|y|} p(k) \log_2 p(k)
$$

In [78]:
# 计算信息熵
def entropy(y):
    unique_classes, counts = np.unique(y, return_counts=True)
    probabilities = counts / len(y)
    return -np.sum(probabilities * np.log2(probabilities))



# 计算信息增益
$$
Gain(D,a) = Ent(D) - \sum_{v=1}^{V}\frac{|D^v|}{|D|}Ent(D^v)
$$


In [79]:
# 计算信息增益
def information_gain(X, y, feature):
    unique_values = np.unique(X[feature])
    weighted_entropy = 0
    for value in unique_values:
        subset_y = y[X[feature] == value]
        weighted_entropy += (len(subset_y) / len(y)) * entropy(subset_y)
    return entropy(y) - weighted_entropy

In [80]:
# 构建ID3决策树
def id3(X, y, features):
    # 如果所有样本的标签相同，返回该标签
    if len(np.unique(y)) == 1:
        return np.unique(y)[0]
    
    # 如果没有特征可供选择，返回出现次数最多的标签
    if len(features) == 0:
        return np.bincount(y).argmax()
    
    # 计算每个特征的信息增益
    gains = [information_gain(X, y, feature) for feature in features]
    
    # 打印
    for feature, gain in zip(features, gains):
        print(f"information_gain on {feature}: {gain}")
    
    # 选择信息增益最大的特征作为当前节点的最佳分裂特征
    best_feature = features[np.argmax(gains)]
    
    print(f"Best feature to split on: {best_feature}")
    
    # 创建一个以最佳特征为根节点的树
    tree = {best_feature: {}}
    # 剩余的特征，不包括最佳特征
    remaining_features = [f for f in features if f != best_feature]
    
    # 对最佳特征的每个唯一值，递归构建子树
    for value in np.unique(X[best_feature]):
        # 根据最佳特征的值划分数据集
        subset_X = X[X[best_feature] == value]
        subset_y = y[X[best_feature] == value]
        # 递归调用id3函数构建子树
        subtree = id3(subset_X, subset_y, remaining_features)
        # 将子树添加到树的相应位置
        tree[best_feature][value] = subtree
    
    # 返回构建的树
    return tree


In [81]:
# 构建ID3决策树
features = X.columns.tolist()
tree = id3(X, y, features)
print(tree)

information_gain on 色泽: 0.10812516526536531
information_gain on 根蒂: 0.14267495956679288
information_gain on 敲声: 0.14078143361499584
information_gain on 纹理: 0.3805918973682686
information_gain on 脐部: 0.28915878284167895
information_gain on 触感: 0.006046489176565584
Best feature to split on: 纹理
information_gain on 色泽: 0.04306839587828004
information_gain on 根蒂: 0.45810589515712374
information_gain on 敲声: 0.33085622540971754
information_gain on 脐部: 0.45810589515712374
information_gain on 触感: 0.45810589515712374
Best feature to split on: 根蒂
information_gain on 色泽: 0.2516291673878229
information_gain on 敲声: 0.0
information_gain on 脐部: 0.0
information_gain on 触感: 0.2516291673878229
Best feature to split on: 色泽
information_gain on 敲声: 0.0
information_gain on 脐部: 0.0
information_gain on 触感: 1.0
Best feature to split on: 触感
information_gain on 色泽: 0.3219280948873623
information_gain on 根蒂: 0.07290559532005603
information_gain on 敲声: 0.3219280948873623
information_gain on 脐部: 0.17095059445466865


In [82]:
# 可视化
graph = visualize_tree(tree)
graph.render("ID3_Decision_Tree_Handwritten")

'ID3_Decision_Tree_Handwritten.png'

## C4.5

# 计算信息增益率
$$
Gain_{ratio}(D,a) = \frac{Gain(D,a)}{IV(a)}
$$

$$
IV(a)=-\sum_{v=1}^{V}\frac{|D^v|}{|D|}\log_{2}\frac{|D^v|}{|D|}
$$

In [83]:
# 计算信息增益率
def information_gain_ratio(X, y, feature):
    unique_values = np.unique(X[feature])
    weighted_entropy = 0
    split_info = 0
    for value in unique_values:
        subset_y = y[X[feature] == value]
        weighted_entropy += (len(subset_y) / len(y)) * entropy(subset_y)
        split_info -= (len(subset_y) / len(y)) * np.log2(len(subset_y) / len(y))
    return (entropy(y) - weighted_entropy) / split_info if split_info != 0 else 0

In [84]:
def c45(X, y, features):
    # 如果所有标签都相同，返回该标签
    if len(np.unique(y)) == 1:
        return np.unique(y)[0]
    
    # 如果没有特征可供选择，返回出现次数最多的标签
    if len(features) == 0:
        return np.bincount(y).argmax()
    
    # 计算每个特征的信息增益率
    gain_ratios = [information_gain_ratio(X, y, feature) for feature in features]
    
    for gain_ratio, feature in zip(gain_ratios, features):
        print(f"information_gain_ratio on {feature}: {gain_ratio}")
    
    # 计算信息增益
    information_gains = [information_gain(X, y, feature) for feature in features]
    for infor_gain, feature in zip(information_gains, features):
        print(f"information_gain on {feature}: {infor_gain}")
    avg_information_gain = np.mean(information_gains)
    print(f"avg_information_gain is {avg_information_gain}")
    
    # 选择信息增益高于平均水平的特征
    valid_features = [features[i] for i in range(len(features)) if information_gains[i] > avg_information_gain]
    
    if not valid_features:
        # 如果没有特征的信息增益高于平均水平，选择信息增益最高的特征
        best_feature = features[np.argmax(information_gains)]
    else:
        # 从信息增益高于平均水平的特征中选择增益率最高的特征
        valid_gain_ratios = [information_gain_ratio(X, y, feature) for feature in valid_features]
        best_feature = valid_features[np.argmax(valid_gain_ratios)]
    
    print(f"Best feature to split on: {best_feature}")    
    
    # 创建一个新的树节点，以最佳特征为根节点
    tree = {best_feature: {}}
    
    # 剩余的特征，不包括最佳特征
    remaining_features = [f for f in features if f != best_feature]
    
    # 对最佳特征的每一个唯一值，递归构建子树
    for value in np.unique(X[best_feature]):
        # 获取当前特征值对应的数据子集
        subset_X = X[X[best_feature] == value]
        subset_y = y[X[best_feature] == value]
        
        # 递归调用c45函数构建子树
        subtree = c45(subset_X, subset_y, remaining_features)
        
        # 将子树添加到树的当前节点
        tree[best_feature][value] = subtree
    
    # 返回构建好的树
    return tree

# 构建C4.5决策树
tree_c45 = c45(X, y, features)
print(tree_c45)

information_gain_ratio on 色泽: 0.06843956584615814
information_gain_ratio on 根蒂: 0.1017593980537369
information_gain_ratio on 敲声: 0.10562670944314426
information_gain_ratio on 纹理: 0.2630853587192754
information_gain_ratio on 脐部: 0.1867268991844879
information_gain_ratio on 触感: 0.0069183298534003
information_gain on 色泽: 0.10812516526536531
information_gain on 根蒂: 0.14267495956679288
information_gain on 敲声: 0.14078143361499584
information_gain on 纹理: 0.3805918973682686
information_gain on 脐部: 0.28915878284167895
information_gain on 触感: 0.006046489176565584
avg_information_gain is 0.17789645463894455
Best feature to split on: 纹理
information_gain_ratio on 色泽: 0.030936667578096773
information_gain_ratio on 根蒂: 0.3389249359511584
information_gain_ratio on 敲声: 0.27022029269335
information_gain_ratio on 脐部: 0.3389249359511584
information_gain_ratio on 触感: 0.49886526560234923
information_gain on 色泽: 0.04306839587828004
information_gain on 根蒂: 0.45810589515712374
information_gain on 敲声: 0.3308562

In [85]:
# 可视化
graph = visualize_tree(tree_c45)
graph.render("C4.5_Decision_Tree_Handwritten")

'C4.5_Decision_Tree_Handwritten.png'

## CART

$$
Gini(D^v) = 1 - \sum_{k=1}^{|y|}p^2_k
$$

In [86]:
# 计算Gini指数
def gini(y):
    unique_classes, counts = np.unique(y, return_counts=True)
    probabilities = counts / len(y)
    return 1 - np.sum(probabilities ** 2)

$$
Gini_{index}(D) = \sum_{v=1}^{|V|}\frac{|D^v|}{|D|}Gini(D^v)
$$

In [87]:
# 计算特征A在取值a时的数据集D的基尼指数
def gini_index(X, y, feature):
    unique_values = np.unique(X[feature])
    weighted_gini = 0
    for value in unique_values:
        subset_y = y[X[feature] == value]
        gini_value = gini(subset_y)
        weighted_gini += (len(subset_y) / len(y)) * gini_value
        print(f"Gini for {feature} = {value}: {gini_value}")
    print(f"Weighted Gini for {feature}: {weighted_gini}")
    return weighted_gini

In [88]:
# 构建CART决策树
def cart(X, y, features, depth=0):
    # 缩进用于打印树的层次结构
    indent = "  " * depth
    
    # 如果所有标签都相同，返回该标签
    if len(np.unique(y)) == 1:
        print(f"{indent}Leaf: {np.unique(y)}")
        return np.unique(y)[0]
    
    # 如果没有特征可供选择，返回出现次数最多的标签
    if len(features) == 0:
        majority_class = np.bincount(y).argmax()
        print(f"{indent}Leaf: {majority_class}")
        return majority_class
    
    # 打印当前特征的Gini指数计算信息
    print(f"{indent}Calculating Gini indices for features: {features}")
    
    # 计算每个特征的Gini指数
    gini_indices = [gini_index(X, y, feature) for feature in features]
    
    # 选择Gini指数最小的特征
    best_feature = features[np.argmin(gini_indices)]
    
    # 打印选择的最佳特征
    print(f"{indent}Best feature to split: {best_feature}")
    
    # 创建一个新的树节点，以最佳特征为根节点
    tree = {best_feature: {}}
    
    # 剩余的特征，不包括最佳特征
    remaining_features = [f for f in features if f != best_feature]
    
    # 对最佳特征的每一个唯一值，递归构建子树
    for value in np.unique(X[best_feature]):
        # 获取当前特征值对应的数据子集
        subset_X = X[X[best_feature] == value]
        subset_y = y[X[best_feature] == value]
        
        # 打印当前分裂的信息
        print(f"{indent}Splitting on {best_feature} = {value}")
        
        # 递归调用cart函数构建子树
        subtree = cart(subset_X, subset_y, remaining_features, depth + 1)
        
        # 将子树添加到树的当前节点
        tree[best_feature][value] = subtree
    
    # 返回构建好的树
    return tree

# 示例数据
X = df.drop('好瓜', axis=1)  # 删除标签列，保留特征
y = df['好瓜'].apply(lambda x: 1 if x == '是' else 0)  # 将标签转换为二进制值
features = X.columns.tolist()  # 获取特征列表

# 构建CART决策树
tree_cart = cart(X, y, features)
print(tree_cart)


Calculating Gini indices for features: ['色泽', '根蒂', '敲声', '纹理', '脐部', '触感']
Gini for 色泽 = 乌黑: 0.4444444444444444
Gini for 色泽 = 浅白: 0.31999999999999984
Gini for 色泽 = 青绿: 0.5
Weighted Gini for 色泽: 0.4274509803921568
Gini for 根蒂 = 硬挺: 0.0
Gini for 根蒂 = 稍蜷: 0.48979591836734704
Gini for 根蒂 = 蜷缩: 0.46875
Weighted Gini for 根蒂: 0.42226890756302526
Gini for 敲声 = 沉闷: 0.48
Gini for 敲声 = 浊响: 0.48
Gini for 敲声 = 清脆: 0.0
Weighted Gini for 敲声: 0.4235294117647059
Gini for 纹理 = 模糊: 0.0
Gini for 纹理 = 清晰: 0.345679012345679
Gini for 纹理 = 稍糊: 0.31999999999999984
Weighted Gini for 纹理: 0.2771241830065359
Gini for 脐部 = 凹陷: 0.40816326530612246
Gini for 脐部 = 平坦: 0.0
Gini for 脐部 = 稍凹: 0.5
Weighted Gini for 脐部: 0.3445378151260504
Gini for 触感 = 硬滑: 0.5
Gini for 触感 = 软粘: 0.48
Weighted Gini for 触感: 0.49411764705882355
Best feature to split: 纹理
Splitting on 纹理 = 模糊
  Leaf: [0]
Splitting on 纹理 = 清晰
  Calculating Gini indices for features: ['色泽', '根蒂', '敲声', '脐部', '触感']
Gini for 色泽 = 乌黑: 0.375
Gini for 色泽 = 浅白: 0.0
Gini

In [89]:
graph = visualize_tree(tree_cart)
graph.render("Cart_Decision_Tree_Handwritten")

'Cart_Decision_Tree_Handwritten.png'

# 3 实现决策树的预剪枝与后剪枝（后剪枝选做）

同样需要文字输出每一步的结果与原因，具体实验步骤如下：

（1）	数据集准备（自行划分训练集与验证集，参照教材表4.2）

（2）	预剪枝函数，需输出每一步剪枝的原因（参照教材图4.6）

（3）	后剪枝函数（选做），需输出每一步剪枝的原因（参照教材图4.6）

（4）	main：构建决策树，并将构建结果写入.dot文件中

（5）	可视化，包含未减枝，预剪枝，后剪枝（选做）

# 划分数据集

In [90]:
# 训练集和验证集的索引
train_indices = [0, 1, 2, 5, 6, 9, 13, 14, 15, 16]
val_indices = [3, 4, 7, 8, 10, 11, 12]

# 划分训练集和验证集
train_set = df.iloc[train_indices]
val_set = df.iloc[val_indices]

print("训练集:")
print(train_set)
print("\n验证集:")
print(val_set)

训练集:
    色泽  根蒂  敲声  纹理  脐部  触感 好瓜
0   青绿  蜷缩  浊响  清晰  凹陷  硬滑  是
1   乌黑  蜷缩  沉闷  清晰  凹陷  硬滑  是
2   乌黑  蜷缩  浊响  清晰  凹陷  硬滑  是
5   青绿  稍蜷  浊响  清晰  稍凹  软粘  是
6   乌黑  稍蜷  浊响  稍糊  稍凹  软粘  是
9   青绿  硬挺  清脆  清晰  平坦  软粘  否
13  浅白  稍蜷  沉闷  稍糊  凹陷  硬滑  否
14  乌黑  稍蜷  浊响  清晰  稍凹  软粘  否
15  浅白  蜷缩  浊响  模糊  平坦  硬滑  否
16  青绿  蜷缩  沉闷  稍糊  稍凹  硬滑  否

验证集:
    色泽  根蒂  敲声  纹理  脐部  触感 好瓜
3   青绿  蜷缩  沉闷  清晰  凹陷  硬滑  是
4   浅白  蜷缩  浊响  清晰  凹陷  硬滑  是
7   乌黑  稍蜷  浊响  清晰  稍凹  硬滑  是
8   乌黑  稍蜷  沉闷  稍糊  稍凹  硬滑  否
10  浅白  硬挺  清脆  模糊  平坦  硬滑  否
11  浅白  蜷缩  浊响  模糊  平坦  软粘  否
12  青绿  稍蜷  浊响  稍糊  凹陷  硬滑  否


In [91]:
# 特征和标签
X_train = train_set.drop('好瓜', axis=1)
y_train = train_set['好瓜'].apply(lambda x: 1 if x == '是' else 0)
X_val = val_set.drop('好瓜', axis=1)
y_val = val_set['好瓜'].apply(lambda x: 1 if x == '是' else 0)

# 训练集上的cart

In [92]:
# 这样子做出来和书上会一样
features = ['脐部', '色泽', '根蒂', '敲声', '纹理', '触感']

In [93]:
# 构建CART决策树
tree_cart = cart(X_train, y_train, features)
print(tree_cart)

Calculating Gini indices for features: ['脐部', '色泽', '根蒂', '敲声', '纹理', '触感']
Gini for 脐部 = 凹陷: 0.375
Gini for 脐部 = 平坦: 0.0
Gini for 脐部 = 稍凹: 0.5
Weighted Gini for 脐部: 0.35000000000000003
Gini for 色泽 = 乌黑: 0.375
Gini for 色泽 = 浅白: 0.0
Gini for 色泽 = 青绿: 0.5
Weighted Gini for 色泽: 0.35000000000000003
Gini for 根蒂 = 硬挺: 0.0
Gini for 根蒂 = 稍蜷: 0.5
Gini for 根蒂 = 蜷缩: 0.48
Weighted Gini for 根蒂: 0.44
Gini for 敲声 = 沉闷: 0.4444444444444444
Gini for 敲声 = 浊响: 0.4444444444444444
Gini for 敲声 = 清脆: 0.0
Weighted Gini for 敲声: 0.4
Gini for 纹理 = 模糊: 0.0
Gini for 纹理 = 清晰: 0.4444444444444444
Gini for 纹理 = 稍糊: 0.4444444444444444
Weighted Gini for 纹理: 0.4
Gini for 触感 = 硬滑: 0.5
Gini for 触感 = 软粘: 0.5
Weighted Gini for 触感: 0.5
Best feature to split: 脐部
Splitting on 脐部 = 凹陷
  Calculating Gini indices for features: ['色泽', '根蒂', '敲声', '纹理', '触感']
Gini for 色泽 = 乌黑: 0.0
Gini for 色泽 = 浅白: 0.0
Gini for 色泽 = 青绿: 0.0
Weighted Gini for 色泽: 0.0
Gini for 根蒂 = 稍蜷: 0.0
Gini for 根蒂 = 蜷缩: 0.0
Weighted Gini for 根蒂: 0.0
Gini for 敲声 = 沉

In [94]:
# 可视化
graph = visualize_tree(tree_cart)
graph.render("tree_cart_train")

'tree_cart_train.png'

# 预剪枝

In [101]:
import numpy as np
import pandas as pd

# 构建CART决策树
def pre_prune_cart(X, y, features, validation_X, validation_y, depth=0):
    # 缩进用于打印树的层次结构
    indent = "  " * depth
    
    # 如果所有标签都相同，返回该标签
    if len(np.unique(y)) == 1:
        print(f"{indent}Leaf: {np.unique(y)}")
        return np.unique(y)
    
    # 如果没有特征可供选择，返回出现次数最多的标签
    if len(features) == 0:
        majority_class = np.bincount(y).argmax()
        print(f"{indent}Leaf: {majority_class}")
        return majority_class
    
    # 打印当前特征的信息增益计算信息
    print(f"{indent}Calculating information gain for features: {features}")
    
    # 计算每个特征的Gini指数
    gini_indexs = [gini_index(X, y, feature) for feature in features]
    
    # 选择Gini指数最小的特征
    best_feature = features[np.argmin(gini_indexs)]
    
    # 预剪枝判断
    # 算当前的，就是当成一个0或者是1到的叶子节点
    majority_class = np.bincount(y).argmax()
    current_accuracy = np.mean(validation_y == majority_class)
    print(f"{indent}Current validation accuracy: {current_accuracy}")
    
    # 计算划分后的验证集精度
    correct_predictions = 0
    # 遍历最佳特征的每一个唯一值
    for value in np.unique(X[best_feature]):
        # subset_X 和 subset_y 是训练集的子集，包含当前特征值等于 value 的样本。
        # subset_validation_X 和 subset_validation_y 是验证集的子集，包含当前特征值等于 value 的样本。
        # 我这边直接用对应节点的属性部分来做。
        subset_X = X[X[best_feature] == value]
        subset_y = y[X[best_feature] == value]
        subset_validation_y = validation_y[validation_X[best_feature] == value]
        if len(subset_y) > 0:
            # 按最多的判别为0或者1
            majority_class = np.bincount(subset_y).argmax()
            correct_predictions += np.sum(subset_validation_y == majority_class)
    
    new_accuracy = correct_predictions / len(validation_y)
    print(f"{indent}New validation accuracy after split: {new_accuracy}")
    
    # 如果新的验证集精度不高于当前精度，则不进行分裂，创建叶节点
    if new_accuracy <= current_accuracy:
        majority_class = np.bincount(y).argmax()
        print(f"{indent}Pre-pruning decision: Do not split, create leaf with class {majority_class}")
        return majority_class
    
    # 打印选择的最佳特征
    print(f"{indent}Best feature to split: {best_feature}")
    
    # 创建一个新的树节点，以最佳特征为根节点
    tree = {best_feature: {}}
    
    # 剩余的特征，不包括最佳特征
    remaining_features = [f for f in features if f != best_feature]
    
    # 对最佳特征的每一个唯一值，递归构建子树
    for value in np.unique(X[best_feature]):
        subset_X = X[X[best_feature] == value]
        subset_y = y[X[best_feature] == value]
        subset_validation_X = validation_X[validation_X[best_feature] == value]
        subset_validation_y = validation_y[validation_X[best_feature] == value]
        
        # 打印当前分裂的信息
        print(f"{indent}Splitting on {best_feature} = {value}")
        print(f"{indent}subset_X:\n{subset_X}")
        print(f"{indent}subset_y:\n{subset_y}")
        print(f"{indent}subset_validation_X:\n{subset_validation_X}")
        print(f"{indent}subset_validation_y:\n{subset_validation_y}")
        
        # 递归调用pre_prune_cart函数构建子树
        subtree = pre_prune_cart(subset_X, subset_y, remaining_features, subset_validation_X, subset_validation_y, depth + 1)
        
        # 将子树添加到树的当前节点
        tree[best_feature][value] = subtree
    
    # 返回构建好的树
    return tree

# 构建CART决策树
tree_cart = pre_prune_cart(X_train, y_train, features, X_val, y_val)
print(tree_cart)


Calculating information gain for features: ['脐部', '色泽', '根蒂', '敲声', '纹理', '触感']
Gini for 脐部 = 凹陷: 0.375
Gini for 脐部 = 平坦: 0.0
Gini for 脐部 = 稍凹: 0.5
Weighted Gini for 脐部: 0.35000000000000003
Gini for 色泽 = 乌黑: 0.375
Gini for 色泽 = 浅白: 0.0
Gini for 色泽 = 青绿: 0.5
Weighted Gini for 色泽: 0.35000000000000003
Gini for 根蒂 = 硬挺: 0.0
Gini for 根蒂 = 稍蜷: 0.5
Gini for 根蒂 = 蜷缩: 0.48
Weighted Gini for 根蒂: 0.44
Gini for 敲声 = 沉闷: 0.4444444444444444
Gini for 敲声 = 浊响: 0.4444444444444444
Gini for 敲声 = 清脆: 0.0
Weighted Gini for 敲声: 0.4
Gini for 纹理 = 模糊: 0.0
Gini for 纹理 = 清晰: 0.4444444444444444
Gini for 纹理 = 稍糊: 0.4444444444444444
Weighted Gini for 纹理: 0.4
Gini for 触感 = 硬滑: 0.5
Gini for 触感 = 软粘: 0.5
Weighted Gini for 触感: 0.5
Current validation accuracy: 0.5714285714285714
New validation accuracy after split: 0.7142857142857143
Best feature to split: 脐部
Splitting on 脐部 = 凹陷
subset_X:
    色泽  根蒂  敲声  纹理  脐部  触感
0   青绿  蜷缩  浊响  清晰  凹陷  硬滑
1   乌黑  蜷缩  沉闷  清晰  凹陷  硬滑
2   乌黑  蜷缩  浊响  清晰  凹陷  硬滑
13  浅白  稍蜷  沉闷  稍糊  凹陷 

In [30]:
# 可视化
graph = visualize_tree(tree_cart)
graph.render("pre_prune")

'pre_prune.png'

# 后剪枝（选做）

In [31]:
# 从左往右遍历
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 后剪枝函数
def post_prune(tree, X_val, y_val):
    def prune_node(node, X, y):
        if not isinstance(node, dict):
            return node

        feature = list(node.keys())[0]
        for value in sorted(node[feature].keys()):  # 从左往右遍历
            subset_X = X[X[feature] == value]
            subset_y = y[X[feature] == value]
            node[feature][value] = prune_node(node[feature][value], subset_X, subset_y)

        # 检查是否所有子节点都是叶节点
        leaf_values = [subtree for subtree in node[feature].values() if not isinstance(subtree, dict)]
        if len(leaf_values) == len(node[feature]):
            # 计算剪枝前后的精确度
            print(f"processing: {feature}")

            y_pred_before = predict(tree, X_val)
            accuracy_before = accuracy_score(y_val, y_pred_before)
            print(f"Accuracy before pruning: {accuracy_before}")

            # 临时移除节点并计算新的预测准确性
            majority_class = np.bincount(y).argmax()
            temp_node = node.copy()
            node[feature] = {value: majority_class for value in node[feature]}
            y_pred_after = predict(tree, X_val)
            accuracy_after = accuracy_score(y_val, y_pred_after)
            print(f"Accuracy after pruning: {accuracy_after}")

            if accuracy_after > accuracy_before:
                print(f"Pruning performed at node: {feature}")
                return majority_class
            else:
                node = temp_node  # 恢复原始节点
                print("not need to prune")

        return node

    return prune_node(tree, X_val, y_val)

# 预测函数
def predict(tree, X):
    def predict_single(tree, x):
        if not isinstance(tree, dict):
            return tree
        feature = list(tree.keys())[0]
        value = x[feature]
        subtree = tree[feature].get(value, 1)  # 缺失值视为好瓜
        return predict_single(subtree, x)

    return X.apply(lambda x: predict_single(tree, x), axis=1)

# 构建CART决策树
tree_cart = cart(X_train, y_train, features)
print("Initial tree:", tree_cart)

# 后剪枝
pruned_tree = post_prune(tree_cart, X_val, y_val)
print("Pruned tree:", pruned_tree)

Calculating Gini indices for features: ['脐部', '色泽', '根蒂', '敲声', '纹理', '触感']
Gini for 脐部 = 凹陷: 0.375
Gini for 脐部 = 平坦: 0.0
Gini for 脐部 = 稍凹: 0.5
Weighted Gini for 脐部: 0.35000000000000003
Gini for 色泽 = 乌黑: 0.375
Gini for 色泽 = 浅白: 0.0
Gini for 色泽 = 青绿: 0.5
Weighted Gini for 色泽: 0.35000000000000003
Gini for 根蒂 = 硬挺: 0.0
Gini for 根蒂 = 稍蜷: 0.5
Gini for 根蒂 = 蜷缩: 0.48
Weighted Gini for 根蒂: 0.44
Gini for 敲声 = 沉闷: 0.4444444444444444
Gini for 敲声 = 浊响: 0.4444444444444444
Gini for 敲声 = 清脆: 0.0
Weighted Gini for 敲声: 0.4
Gini for 纹理 = 模糊: 0.0
Gini for 纹理 = 清晰: 0.4444444444444444
Gini for 纹理 = 稍糊: 0.4444444444444444
Weighted Gini for 纹理: 0.4
Gini for 触感 = 硬滑: 0.5
Gini for 触感 = 软粘: 0.5
Weighted Gini for 触感: 0.5
Best feature to split: 脐部
Splitting on 脐部 = 凹陷
  Calculating Gini indices for features: ['色泽', '根蒂', '敲声', '纹理', '触感']
Gini for 色泽 = 乌黑: 0.0
Gini for 色泽 = 浅白: 0.0
Gini for 色泽 = 青绿: 0.0
Weighted Gini for 色泽: 0.0
Gini for 根蒂 = 稍蜷: 0.0
Gini for 根蒂 = 蜷缩: 0.0
Weighted Gini for 根蒂: 0.0
Gini for 敲声 = 沉

In [32]:
# 更书上一样的做法

# 计算树的最大深度
def max_depth(tree):
    if not isinstance(tree, dict):
        return 0
    feature = list(tree.keys())[0]
    return 1 + max(max_depth(subtree) for subtree in tree[feature].values())

# 后剪枝函数
def post_prune(tree, X_val, y_val):
    def prune_node(node, X, y, current_depth, max_depth):
        if not isinstance(node, dict):
            return node

        feature = list(node.keys())[0]
        if current_depth < max_depth:
            for value in sorted(node[feature].keys()):  # 从左往右遍历
                subset_X = X[X[feature] == value]
                subset_y = y[X[feature] == value]
                node[feature][value] = prune_node(node[feature][value], subset_X, subset_y, current_depth + 1, max_depth)
        else:
            # 检查是否所有子节点都是叶节点
            leaf_values = [subtree for subtree in node[feature].values() if not isinstance(subtree, dict)]
            if len(leaf_values) == len(node[feature]):
                # 计算剪枝前后的精确度
                print(f"processing: {feature}")

                y_pred_before = predict(tree, X_val)
                accuracy_before = accuracy_score(y_val, y_pred_before)
                print(f"Accuracy before pruning: {accuracy_before}")

                # 临时移除节点并计算新的预测准确性
                majority_class = np.bincount(y).argmax()
                temp_node = node.copy()
                node[feature] = {value: majority_class for value in node[feature]}
                y_pred_after = predict(tree, X_val)
                accuracy_after = accuracy_score(y_val, y_pred_after)
                print(f"Accuracy after pruning: {accuracy_after}")

                if accuracy_after > accuracy_before:
                    print(f"Pruning performed at node: {feature}")
                    return majority_class
                else:
                    node = temp_node  # 恢复原始节点
                    print("not need to prune")

        return node

    def prune_tree(tree, X, y, max_depth):
        for depth in range(max_depth, 0, -1):
            tree = prune_node(tree, X, y, 1, depth)
        return tree

    max_depth_value = max_depth(tree)
    previous_tree = None
    while previous_tree != tree:
        previous_tree = tree.copy()
        tree = prune_tree(tree, X_val, y_val, max_depth_value)
        # 检查并处理其他层次的节点
        for depth in range(max_depth_value - 1, 0, -1):
            tree = prune_tree(tree, X_val, y_val, depth)
    return tree

# 预测函数
def predict(tree, X):
    def predict_single(tree, x):
        if not isinstance(tree, dict):
            return tree
        feature = list(tree.keys())[0]
        value = x[feature]
        subtree = tree[feature].get(value, 1)  # 缺失值视为好瓜
        return predict_single(subtree, x)

    return X.apply(lambda x: predict_single(tree, x), axis=1)

# 构建CART决策树
tree_cart = cart(X_train, y_train, features)
print("Initial tree:", tree_cart)

# 后剪枝
pruned_tree = post_prune(tree_cart, X_val, y_val)
print("Pruned tree:", pruned_tree)

Calculating Gini indices for features: ['脐部', '色泽', '根蒂', '敲声', '纹理', '触感']
Gini for 脐部 = 凹陷: 0.375
Gini for 脐部 = 平坦: 0.0
Gini for 脐部 = 稍凹: 0.5
Weighted Gini for 脐部: 0.35000000000000003
Gini for 色泽 = 乌黑: 0.375
Gini for 色泽 = 浅白: 0.0
Gini for 色泽 = 青绿: 0.5
Weighted Gini for 色泽: 0.35000000000000003
Gini for 根蒂 = 硬挺: 0.0
Gini for 根蒂 = 稍蜷: 0.5
Gini for 根蒂 = 蜷缩: 0.48
Weighted Gini for 根蒂: 0.44
Gini for 敲声 = 沉闷: 0.4444444444444444
Gini for 敲声 = 浊响: 0.4444444444444444
Gini for 敲声 = 清脆: 0.0
Weighted Gini for 敲声: 0.4
Gini for 纹理 = 模糊: 0.0
Gini for 纹理 = 清晰: 0.4444444444444444
Gini for 纹理 = 稍糊: 0.4444444444444444
Weighted Gini for 纹理: 0.4
Gini for 触感 = 硬滑: 0.5
Gini for 触感 = 软粘: 0.5
Weighted Gini for 触感: 0.5
Best feature to split: 脐部
Splitting on 脐部 = 凹陷
  Calculating Gini indices for features: ['色泽', '根蒂', '敲声', '纹理', '触感']
Gini for 色泽 = 乌黑: 0.0
Gini for 色泽 = 浅白: 0.0
Gini for 色泽 = 青绿: 0.0
Weighted Gini for 色泽: 0.0
Gini for 根蒂 = 稍蜷: 0.0
Gini for 根蒂 = 蜷缩: 0.0
Weighted Gini for 根蒂: 0.0
Gini for 敲声 = 沉

In [33]:
# 可视化
graph = visualize_tree(pruned_tree)
graph.render("post_prune")

'post_prune.png'